In [2]:
import numpy as np
import pandas as pd
import xgboost as xgb

In [3]:
df = pd.read_csv("data_semantics_training.csv")

In [4]:
df.drop(["serial_no"],axis=1,inplace=True)

In [5]:
categorical = df.select_dtypes(include=['object'])
cat_values=categorical.columns.values
for i in range(len(cat_values)):
    print('For the column "',cat_values[i],'" the unique values are')
    print(df[cat_values[i]].unique())

For the column " C7 " the unique values are
['Healthy' 'Defaulted' 'None' 'Unhealthy']
For the column " C8 " the unique values are
['Live' 'Closed' nan]


In [6]:
df=df.fillna(method='ffill')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 699 entries, 0 to 698
Data columns (total 8 columns):
C1    699 non-null int64
C2    699 non-null int64
C3    699 non-null int64
C4    699 non-null int64
C5    699 non-null float64
C6    699 non-null float64
C7    699 non-null object
C8    699 non-null object
dtypes: float64(2), int64(4), object(2)
memory usage: 43.8+ KB


In [8]:
y=df['C7']
y=pd.DataFrame(y)
X=df.drop(['C7'],axis=1)

In [9]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 699 entries, 0 to 698
Data columns (total 7 columns):
C1    699 non-null int64
C2    699 non-null int64
C3    699 non-null int64
C4    699 non-null int64
C5    699 non-null float64
C6    699 non-null float64
C8    699 non-null object
dtypes: float64(2), int64(4), object(1)
memory usage: 38.3+ KB


In [10]:
y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 699 entries, 0 to 698
Data columns (total 1 columns):
C7    699 non-null object
dtypes: object(1)
memory usage: 5.5+ KB


In [11]:
X=pd.get_dummies(X,columns = ['C8'])
y['C7'] = y['C7'].map({'Healthy':0, 'Unhealthy':1,'None':2,'Unhealthy':3})


In [12]:
X.head()

,C1,C2,C3,C4,C5,C6,C8_Closed,C8_Live
0,1,1,15200,80,6.50,24.0,0,1
1,2,2,26100,90,6.75,30.0,1,0
2,3,3,22400,80,7.50,36.0,1,0
3,4,4,21600,90,7.50,42.0,1,0
4,5,5,44000,100,10.00,60.0,0,1


In [13]:
y=y.fillna(method='ffill')

In [14]:
y.describe()

,C7
count,699.000000
mean,0.273247
std,0.761549
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,3.000000


In [15]:
from sklearn.model_selection import train_test_split

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y,random_state=123)

In [17]:
alg = xgb.XGBClassifier(learning_rate=0.1, n_estimators=140, max_depth=5,
                        min_child_weight=3, gamma=0.2, subsample=0.6, colsample_bytree=1.0,
                        objective='binary:logistic', nthread=4, scale_pos_weight=1, seed=27)

In [18]:
model = alg.fit(X,y)

/home/khandalaryan/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/khandalaryan/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [19]:
pred1 = model.predict(X_test)

In [20]:
from sklearn.metrics import accuracy_score
print("Accuracy for model 1: %.2f" % (accuracy_score(y_test, pred1) * 100))

Accuracy for model 1: 98.57


In [40]:
model3 = xgb.XGBClassifier(
 learning_rate =0.1,
 n_estimators=250,
 max_depth=5,
 min_child_weight=1,
 gamma=0.1,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'multi:softmax',
 nthread=4,
 scale_pos_weight=1,
 seed=27)

In [41]:
train_model3 = model3.fit(X, y)
pred3 = train_model3.predict(X_test)
print("Accuracy for model 3: %.2f" % (accuracy_score(y_test, pred3) * 100))

Accuracy for model 3: 100.00


In [36]:
from sklearn.model_selection import GridSearchCV

param_test = {
 'n_estimators':np.arange(250,2500,250)
}
gsearch = GridSearchCV(estimator = xgb.XGBClassifier( learning_rate=0.1, n_estimators=140, max_depth=5,
 min_child_weight=2, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'multi:softmax', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test,n_jobs=4,iid=False, cv=5)

train_model4 = gsearch.fit(X_train, y_train)
pred4 = train_model4.predict(X_test)
print("Accuracy for model 4: %.2f" % (accuracy_score(y_test, pred4) * 100))

/home/khandalaryan/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/khandalaryan/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy for model 4: 97.14


In [37]:
gsearch.best_params_

{'n_estimators': 250}

Predicting output

In [55]:
df2 = pd.read_csv("data_sematics_test.csv")

In [56]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301 entries, 0 to 300
Data columns (total 8 columns):
serial_no    301 non-null int64
C1           301 non-null int64
C2           301 non-null int64
C3           301 non-null int64
C4           301 non-null int64
C5           294 non-null float64
C6           294 non-null float64
C8           248 non-null object
dtypes: float64(2), int64(5), object(1)
memory usage: 18.9+ KB


In [57]:
df2.drop(['serial_no'],axis=1,inplace=True)

In [58]:
df2.fillna(method = 'ffill')

,C1,C2,C3,C4,C5,C6,C8
0,700,700,16800,80,6.50,24.0,Closed
1,701,701,0,0,6.50,24.0,Closed
2,702,702,15200,80,6.50,18.0,Closed
3,703,703,14500,50,4.75,18.0,Closed
4,704,704,16800,80,7.75,24.0,Closed
5,705,705,57600,90,11.75,30.0,Live
6,706,706,22400,80,7.25,18.0,Live
7,707,707,15200,80,6.50,18.0,Closed
8,708,708,16800,80,6.75,18.0,Closed
9,709,709,25200,90,7.50,18.0,Closed


In [59]:
df2=pd.get_dummies(df2,columns = ['C8'])

In [60]:
preds = train_model3.predict(df2)

In [48]:
_ = pd.read_csv("data_sematics_test.csv")
serial = _['serial_no']
data = { 'serial_no': serial, 'C7': preds}
submission = pd.DataFrame(data)
submission.head()

,serial_no,C7
0,1,0.0
1,2,2.0
2,3,0.0
3,4,0.0
4,5,0.0


In [49]:
submission['C7'] = submission['C7'].map({0:'Healthy', 1:'Unhealthy',2:'None',3:'Unhealthy'})


In [50]:
submission.to_csv("3xgb.csv",index=False)

Neural Nets

In [52]:
from sklearn.neural_network import MLPClassifier

In [53]:
clf = MLPClassifier()
clf.fit(X, y)                         

/home/khandalaryan/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:916: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [61]:
clf.predict(df2)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').